In [25]:
import json
import pandas as pd

In [26]:
file = './data/ep_ce_instances.json'
f = open(file, 'r')
data = json.load(f)['data']

In [27]:
len(data)

2531

In [28]:
gcp_data = list(filter(lambda x: x.get('provider') == 'GCP', data))

In [29]:
len(gcp_data)

190

In [72]:
ep_gcp_map = {}
for i in range(0, len(gcp_data)):
    variant = gcp_data[i]['variant']
    if variant.get('id'):
#         ep_gcp_map[ep_id] = variant['id']
        ep_gcp_map[variant['id']] = {
            "id": variant['id'],
            "disk_type": list(map(lambda x: x.get('type'), variant['disks']))[0],
            "instanceType": variant["machineType"].split('/')[-1],
            "region": variant.get("region")
        }

In [73]:
gcp_data[0]

{'state': 'STOPPED',
 'status': 'READY',
 'volumes': [],
 'securityGroups': [{'isDefault': True,
   'status': 'RUNNING',
   'provider': 'GCP',
   '_id': '636a869d5456210011eb18f6',
   'name': 'toanntt-gcp-sg-default',
   'description': 'toanntt-gcp-sg-default',
   'owner': 'toanntt',
   'vpcId': '2938434322906074877',
   'createdBy': 'toanntt',
   'createdAt': '2022-11-08T16:41:01.830Z',
   'updatedAt': '2022-11-08T16:43:15.742Z',
   'updatedBy': None,
   'variant': {'GroupId': 'gcp-sg-636a869d5456210011eb18f6',
    'results': {'id': 'gcp-sg-636a869d5456210011eb18f6',
     'virtualmachinecount': 1,
     'virtualmachineids': ['2265529010543344118']},
    'vpcSelfLink': 'https://www.googleapis.com/compute/v1/projects/ts-engineering-337304/global/networks/default',
    'targetTags': ['bot', 'toanntt-gg-cloud']},
   'id': '636a869d5456210011eb18f6'}],
 'provider': 'GCP',
 '_id': '636a87115456210011eb1917',
 'uid': 'toanntt_11',
 'createdBy': 'toanntt',
 'createdAt': '2022-11-08T16:42:57.87

In [74]:
ep_gcp_map

{'2265529010543344118': {'id': '2265529010543344118',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small',
  'region': None},
 '5353556143556258917': {'id': '5353556143556258917',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small',
  'region': None},
 '3922265808357194945': {'id': '3922265808357194945',
  'disk_type': 'PERSISTENT',
  'instanceType': 'f1-micro',
  'region': None},
 '4036654423970744204': {'id': '4036654423970744204',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small',
  'region': None},
 '1240637607697652238': {'id': '1240637607697652238',
  'disk_type': 'PERSISTENT',
  'instanceType': 'f1-micro',
  'region': None},
 '6856595969022927546': {'id': '6856595969022927546',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small',
  'region': None},
 '7363616063745089519': {'id': '7363616063745089519',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small',
  'region': None},
 '3400834221456700298': {'id': '3400834221456700298',
  'disk_type': 'PERSIS

In [41]:
gcp_ce_file = './data/gcp_cost.csv'
df_cost = pd.read_csv(gcp_ce_file, header=0, sep=',', dtype =  {"gcpId": str})

In [42]:
df_cost

,Unnamed: 0,cost,date,costInUsd,gcpId
0,0,0.953786,2022-07-04,0.953786,6762571275092817403
1,1,0.839175,2022-07-04,0.839175,7994828870141789187
2,2,0.211182,2022-09-07,0.211182,2084983531904533635
3,3,0.324464,2022-09-07,0.324464,1037551659627754032
4,4,0.326841,2022-09-07,0.326841,1774086229609597003
...,...,...,...,...,...
5637,5637,0.399614,2022-09-14,0.399614,8842805660917525667
5638,5638,0.834418,2022-08-30,0.834418,7440776697402117547
5639,5639,0.142453,2022-07-27,0.142453,2332843410177307106
5640,5640,2.482112,2022-08-01,2.482112,3838479753825543664


In [65]:
# df['gcpId'] = df.apply(lambda row: str(ep_gcp_map.get(row.epId)), axis=1)
df_new = df_cost
def extract_variant(row, name):
    record = ep_gcp_map.get(row.gcpId)
    if not record:
        return ''
    return record.get(name)
df_new['instanceType'] = df_new.apply(lambda row: extract_variant(row, 'instanceType'), axis=1)
df_new['disk'] = df_new.apply(lambda row: extract_variant(row, 'disk_type'), axis=1)

In [66]:
df_filtered = df_new.loc[df['gcpId'] != None]

In [70]:
df_filtered

,Unnamed: 0,cost,date,costInUsd,gcpId,instanceType,disk
0,0,0.953786,2022-07-04,0.953786,6762571275092817403,g1-small,PERSISTENT
1,1,0.839175,2022-07-04,0.839175,7994828870141789187,g1-small,PERSISTENT
2,2,0.211182,2022-09-07,0.211182,2084983531904533635,g1-small,PERSISTENT
3,3,0.324464,2022-09-07,0.324464,1037551659627754032,f1-micro,PERSISTENT
4,4,0.326841,2022-09-07,0.326841,1774086229609597003,f1-micro,PERSISTENT
...,...,...,...,...,...,...,...
5637,5637,0.399614,2022-09-14,0.399614,8842805660917525667,f1-micro,PERSISTENT
5638,5638,0.834418,2022-08-30,0.834418,7440776697402117547,g1-small,PERSISTENT
5639,5639,0.142453,2022-07-27,0.142453,2332843410177307106,f1-micro,PERSISTENT
5640,5640,2.482112,2022-08-01,2.482112,3838479753825543664,g1-small,PERSISTENT


In [69]:
df_filtered[['cost', 'date', 'costInUsd', 'gcpId', 'instanceType', 'disk']].to_csv('./data/gcp_cost.csv')